Here we will construct the CIs from the multiple trained bootstrap models. We have trained 10 bootstraps for now, we will use these to get the required CIs as we have done earlier for a different dataset.

In [1]:
# These are the ones that is already on the train script
import os
import json
import sys
import warnings
from pathlib import Path
from pprint import pformat
from typing import Dict, Union
import tensorflow as tf

import pickle
import pandas as pd
import numpy as np
from tensorflow.keras import backend as K
# generator related imports
from New_data_generator_with_tf import DataGenerator, BootstrapGenerator, batch_predict
from tensorflow.keras.utils import Sequence

# [Req] IMPROVE imports
# notice that the improvelibs are in the folder that is a level above, but in the same parent directory
sys.path.append(os.path.abspath(os.path.join('..', 'IMPROVE')))
from improvelib.applications.drug_response_prediction.config import DRPTrainConfig
from improvelib.utils import str2bool
import improvelib.utils as frm
from improvelib.metrics import compute_metrics

# Model-specific imports
from model_params_def import train_params # [Req]

2025-04-21 21:15:21.604717: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-21 21:15:22.692891: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# First we compute the model misspecification variance (estimate) using the already trained models using the train dataset.

In [3]:
# For computing the PIs we also need to estimate the variance of the errors. Once we compute the above, we can follow the steps in the paper "Constructing Optimal Predictio Intervals by Using Neural Networks and Bootstrap Method" (we follow their section II and not their proposed method as it seems a little complex to implement)by Khosravi et al. in 2015.

In [4]:
# Let's get started with the first part stated above

In [5]:
# We need the predictions first for the train data - let's get those and later think about what we need next

In [6]:
# We need to arrange our train and validation data for this exercise too

In [7]:
# specify the directory where preprocessed data is stored
data_dir = 'exp_result'

In [8]:
with open(os.path.join(data_dir, "drug_features.pickle"),"rb") as f:
        dict_features = pickle.load(f)

In [9]:
with open(os.path.join(data_dir, "norm_adj_mat.pickle"),"rb") as f:
        dict_adj_mat = pickle.load(f)

In [10]:
train_keep = pd.read_csv(os.path.join(data_dir, "train_y_data.csv"))
valid_keep = pd.read_csv(os.path.join(data_dir, "val_y_data.csv"))

In [11]:
train_keep.shape, valid_keep.shape

((7616, 3), (952, 3))

In [12]:
train_keep.columns = ["Cell_Line", "Drug_ID", "AUC"]
valid_keep.columns = ["Cell_Line", "Drug_ID", "AUC"]

In [13]:
samp_drug = valid_keep["Drug_ID"].unique()[-1]
samp_ach = np.array(valid_keep["Cell_Line"].unique()[-1])

In [14]:
print(samp_drug)
print(samp_ach)

Drug_1326
ACH-000828


In [15]:
train_gcn_feats = []
train_adj_list = []
for drug_id in train_keep["Drug_ID"].values:
    train_gcn_feats.append(dict_features[drug_id])
    train_adj_list.append(dict_adj_mat[drug_id])

In [16]:
len(train_gcn_feats), len(train_adj_list)

(7616, 7616)

In [17]:
valid_gcn_feats = []
valid_adj_list = []
for drug_id in valid_keep["Drug_ID"].values:
    valid_gcn_feats.append(dict_features[drug_id])
    valid_adj_list.append(dict_adj_mat[drug_id])

In [18]:
len(valid_gcn_feats), len(valid_adj_list)

(952, 952)

In [19]:
%%time
# reduce the values to float16
train_gcn_feats = np.array(train_gcn_feats).astype("float32")
valid_gcn_feats = np.array(valid_gcn_feats).astype("float32")

train_adj_list = np.array(train_adj_list).astype("float32")
valid_adj_list = np.array(valid_adj_list).astype("float32")

CPU times: user 509 ms, sys: 779 ms, total: 1.29 s
Wall time: 1.3 s


In [20]:
# Define the data generators for both train and validation datasets. Let's use the train one now, and later think how we can use the validation data generator

In [21]:
train_data_gen = DataGenerator(train_gcn_feats, train_adj_list, train_keep["Cell_Line"].values.reshape(-1,1), train_keep["Cell_Line"].values.reshape(-1,1), train_keep["Cell_Line"].values.reshape(-1,1), train_keep["AUC"].values.reshape(-1,1), batch_size=32,  shuffle = False)

In [22]:
val_data_gen = DataGenerator(valid_gcn_feats, valid_adj_list, valid_keep["Cell_Line"].values.reshape(-1,1), valid_keep["Cell_Line"].values.reshape(-1,1), valid_keep["Cell_Line"].values.reshape(-1,1), valid_keep["AUC"].values.reshape(-1,1), batch_size=32,  shuffle = False)

In [23]:
# Okay, I think now once the model is loaded, we can just get the predictions

In [24]:
%%time
# I don't think we need a function for this, we should be able to just get it done in a for loop

# location of the models
folder_path = 'bootstrap_results_all'

# name of the trained model
model_nm = 'DeepCDR_model'

all_train_predictions = []
train_true = []
# number of boostraps
B = 10

# start the for loop
for i in range(1, B + 1):
    # create the folder
    folder_nm = 'bootstrap_' + str(i)
    # joined path
    folder_loc = os.path.join(folder_path, folder_nm, model_nm)
    # load the model?
    model = tf.keras.models.load_model(folder_loc)
    # get the predictions on the train data
    y_train_preds, y_train_true = batch_predict(model, train_data_gen)
    all_train_predictions.append(y_train_preds)
    train_true.append(y_train_true)

# Notice that we can add some lists to capture the predictions on the validation data as well, eventhough we have these stored, as this will save time - and I think we also need these on the test data inorder the compute the evaluation metrics, like the coverages and the widths.

2025-04-21 21:15:42.361558: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-21 21:15:44.561828: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30960 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0
2025-04-21 21:15:49.430102: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401


Predictions: 7616
True: 7616
Predictions: 7616
True: 7616
Predictions: 7616
True: 7616
Predictions: 7616
True: 7616
Predictions: 7616
True: 7616
Predictions: 7616
True: 7616
Predictions: 7616
True: 7616
Predictions: 7616
True: 7616
Predictions: 7616
True: 7616
Predictions: 7616
True: 7616
CPU times: user 2min 40s, sys: 13 s, total: 2min 53s
Wall time: 2min 39s


In [25]:
all_preds_array = np.array(all_train_predictions)

In [26]:
all_preds_array.shape

(10, 7616)

In [27]:
all_trues = np.array(train_true)

In [28]:
all_trues.shape

(10, 7616)

In [29]:
all_true_mean = np.mean(all_trues, axis = 0)

In [30]:
all_true_mean.shape

(7616,)

In [31]:
all_true_mean

array([0.7153, 0.9579, 0.413 , ..., 0.522 , 0.9436, 0.9835])

In [32]:
np.squeeze(train_keep["AUC"].values.reshape(-1,1))

array([0.7153, 0.9579, 0.413 , ..., 0.522 , 0.9436, 0.9835])

In [33]:
np.squeeze(train_keep["AUC"].values.reshape(-1,1)).shape

(7616,)

In [34]:
# these do look the same, should we do an np.mean?
np.mean(np.round(all_true_mean, 8) == np.round(np.squeeze(train_keep["AUC"].values.reshape(-1,1)), 8))

1.0

In [35]:
# Indeed the y true values match - sanity check complete

In [36]:
# Cool, so what next?

# I think computing the bootstrap quantities, the means and the variance of the predictions. Then we can work on training the NNe

In [37]:
# let's first compute the bootstrap means

train_bts_mean = np.mean(all_preds_array, axis = 0)

In [38]:
train_bts_mean.shape

(7616,)

In [39]:
# we also need the bootstrap variance

# let's use the same function as earlier - we cannot use this as is, as what we have now is a 2D array, and not a 3D one
def equation_6_model_variance(all_preds):
    all_vars = []
    for i in range(all_preds.shape[1]):
        var = (1/(all_preds.shape[0]  - 1))*np.sum(np.square(all_preds[:,i] - np.mean(all_preds[:,i])))
        all_vars.append(var)

    return np.array(all_vars, dtype= np.float32)

In [40]:
train_bts_variance = equation_6_model_variance(all_preds_array)

In [41]:
train_bts_variance.shape

(7616,)

In [42]:
train_bts_variance

array([0.00183184, 0.00109681, 0.00047159, ..., 0.00504019, 0.00060388,
       0.0024684 ], dtype=float32)

In [43]:
# how to alternatively compute the variance in one line
alt_bts_variance = np.var(all_preds_array, axis = 0, ddof = 1)

In [44]:
alt_bts_variance.shape

(7616,)

In [45]:
alt_bts_variance

array([0.00183184, 0.00109681, 0.00047159, ..., 0.00504019, 0.00060388,
       0.0024684 ], dtype=float32)

In [46]:
type(alt_bts_variance), type(train_bts_variance)

(numpy.ndarray, numpy.ndarray)

In [47]:
np.mean(np.round(train_bts_variance, 6) == np.round(alt_bts_variance, 6))

1.0

In [48]:
# sanity check for the means
catch_mean = []
for i in range(all_preds_array.shape[1]):
    computed_mean = np.mean(all_preds_array[:,i])
    catch_mean.append(computed_mean)


In [49]:
sanity_check_means = np.array(catch_mean)
np.mean(np.round(train_bts_mean,2) == np.round(sanity_check_means, 2))

1.0

In [50]:
# Okay, we have the variances and the means, now what?

In [51]:
# We need to compute the rsquare value mentioned in equation 10 of the paper

In [52]:
train_keep["AUC"].values.shape

(7616,)

In [53]:
# Compute r^2(x_i) for each bootstrap model
def compute_r_squared(y_true, y_pred, model_variance):
    residuals = (y_true - y_pred) ** 2 - model_variance
    return np.maximum(residuals, 0)

In [54]:
r_2_true = compute_r_squared(train_keep["AUC"].values, train_bts_mean, train_bts_variance)

In [55]:
r_2_true.shape

(7616,)

In [56]:
r_2_true

array([0.00820237, 0.00362897, 0.00079942, ..., 0.07589278, 0.00014988,
       0.01846294])

In [57]:
# are there any zeros?
np.min(r_2_true)

0.0

In [58]:
# Count the number of zeros
num_zeros = np.count_nonzero(r_2_true == 0)

In [59]:
num_zeros

3852

In [60]:
# That is a lot of zeros, but let's role with this number?

#### We will define the custom loss function in equation 12 and also define a very basic model for the NNe -  Okay, this isn't as straight forward as how we would have liked, so we need to revisit this, and decide what the model structure we need for the NNe. Continue the work on this tomorrow?

In [61]:
# Define the custom loss function as described in equation 12
def correct_custom_loss(r_true, r_pred):
    # first term in equation 12
    term_1 = tf.math.log(r_pred + 1)
    # define the second term
    term_2 = r_true/r_pred
    # cost function
    cost = 0.5 * tf.reduce_mean(term_1 + term_2)

    return cost

In [62]:
# Step 3: Define the noise variance estimation network (Phase II)
def create_noise_variance_nn(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation=tf.keras.activations.exponential)  # Output layer for variance prediction
    ])
    return model

In [63]:
# Train function
def train_model_nne(X_train, y_train, bootstrap_predictions, model_variance, model):
    # Define custom loss inside model.compile using lambda
    
    model.compile(optimizer='adam', 
                  loss=lambda y_true, y_pred: correct_custom_loss(
                      y_true, y_pred))  # y_true corresponds to r_true, and y_pred corresponds to r_pred
    
    bootstrap_mean_predictions = np.squeeze(np.mean(bootstrap_predictions, axis = 0))
    # The true residuals (r_true) can be computed outside and passed during training
    r_true = compute_r_squared(y_train, bootstrap_mean_predictions, model_variance)
    
    # Fit the model with the true residuals
    model.fit(X_train, r_true, epochs=50, batch_size=32)

    return model